In [4]:
import os
import json
import openai
import pandas as pd
import datetime as dt
import streamlit as st
import unidecode as ud
from os.path import exists
from pptx import Presentation
# from pptx.enum.shapes import MSO_SHAPE
# from pptx.dml.color import RGBColor

In [14]:
##############
# Input file locations and names
##############

acn_login = os.getlogin()
folderCV = "C:/Users/" + acn_login + "/Accenture/Warsaw Analytics - Documents"
folderAV = "C:/Users/" + acn_login + "/Accenture/Staffing & productivity AI group - General"
folderApp = "C:/Users/" + acn_login + "/Desktop/genAI/FeedbApp"

CVpptxNm = "/01_CVs/Warsaw_Analytics_FY23_template.pptx"
CVpptxPt = folderCV + CVpptxNm
AVxlsxNm = "/Dashboard/myScheduling_People_Extract.xlsx"
LCRxlsNm = "/Dashboard/Staffing_data - LCR calc.xlsx"
AVxlsxPt = folderAV + AVxlsxNm
LCRxlsPt = folderAV + LCRxlsNm
Sel_txt = folderApp + '/01_CVs/CVapp/sel_list.txt'

In [9]:
# Load configuration from JSON file
with open(folderApp + "/config.json", mode="r") as f:
    config = json.load(f)

client = openai.AzureOpenAI(
        azure_endpoint=config["AZURE_ENDPOINT"],
        api_key= config["AZURE_API_KEY"],
        api_version="2023-12-01-preview")

In [23]:
# Loading input files
CVprs = Presentation(open(CVpptxPt, "rb"))
AVexl = pd.read_excel(AVxlsxPt, 'People_Extract', header=0)
AVexl = AVexl[['Resource Email', 'First Availability Date']]
LCRxl = pd.read_excel(LCRxlsPt, 'LCR', header=0)
LCRxl = LCRxl[['Enterprise ID', 'LCR in $']]

In [ ]:
##############
# Scraping the pptx to produce a table with slidenums, names and positions
##############

CVlist = []
Sel_list = []

for slide in CVprs.slides:
    sldnm = CVprs.slides.index(slide)+1
    if slide.shapes.title != None:
        persn = ud.unidecode(slide.shapes.title.text.title()).strip()
        title = ''
        deptm = ''
        for shape in slide.shapes:
            if (shape.left > 5500000 and shape.left < 10000000 and
                shape.top < 500000):  # locating shape with job title
                if(len(shape.text.split(','))) > 1:
                    title = shape.text.split(',')[0]
                    deptm = shape.text.split(',')[1]
                else:
                    title = shape.text
                    deptm = ''
        if persn not in ['Team Cvs', 'Template']: 
            CVlist.append([sldnm, persn, title, deptm])

CVdf = pd.DataFrame(list(CVlist),
                    columns=['Slide', 'Person', 'Title', 'Dept'])